In [1]:
import datajoint as dj
import numpy as np
import time

import pandas as pd
import seaborn as sns
sns.set() #sets the asethetics parameters in what set

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')
pinky_fc = dj.create_virtual_module("pinky_fc","microns_pinky_fc")

Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class SkeletonCompartmentLengthWithAxon(dj.Computed):
    definition = """
    # Total lengths of each compartment per Skeleton.
    -> pinky.DiscretizedSkeleton
    ---
    apical_length    : float  # Length of the skeleton belonging to the apical compartment.
    basal_length     : float  # Length of the skeleton belonging to the basal compartment.
    oblique_length   : float  # Length of the skeleton belonging to the oblique compartment.
    axon_soma_length   : float  # Length of the skeleton belonging to the axon_soma compartment.
    axon_dendr_length   : float  # Length of the skeleton belonging to the axon_soma compartment.
    dendrites_length : float  # Length of the skeleton belonging to the generic dendrite compartment.
    """

    _compartment_relation = pinky.LabelKey & [dict(numeric=n) for n in [2, 3, 4, 6, 7, 8]]
    _compartment_type_map = {key_map['description'].lower().replace("-","_"): key_map['numeric'] for key_map in _compartment_relation.fetch(as_dict=True)}
    
    key_source = pinky.DiscretizedSkeleton & [dict(class_name='FilteredSkeletonMinusSoma'), dict(class_name='FilteredNeuronSkeleton')]

    def make(self, key):
        skeleton_edges = (pinky.DiscretizedSkeleton & key).fetch1('edges')
        skeleton_labels = (pinky.SkeletonLabel & key).fetch1('edge_midpoint_labels')

        insert_dict = dict(key)
        for description, numeric in self._compartment_type_map.items():
            compartment_edges = skeleton_edges[skeleton_labels==numeric]
            compartment_length = np.linalg.norm(compartment_edges[:, 0] - compartment_edges[:, 1], axis=1).sum()
            insert_dict[f'{description}_length'] = compartment_length
        
        insert_dict['dendrites_length'] = insert_dict['dendrite_length']
        del insert_dict['dendrite_length']

        self.insert1(insert_dict)

In [7]:
#(schema.jobs & "table_name='__skeleton_compartment_length_with_axon'").delete()
#schema.jobs

In [4]:
SkeletonCompartmentLengthWithAxon()

segmentation segmentation id,segment_id segment id unique within each Segmentation,class_name,maximum_edge_length,synaptic_side,apical_length Length of the skeleton belonging to the apical compartment.,basal_length Length of the skeleton belonging to the basal compartment.,oblique_length Length of the skeleton belonging to the oblique compartment.,axon_soma_length Length of the skeleton belonging to the axon_soma compartment.,axon_dendr_length Length of the skeleton belonging to the axon_soma compartment.,dendrites_length Length of the skeleton belonging to the generic dendrite compartment.
3,648518346341353788,FilteredNeuronSkeleton,50,postsyn,0.0,0.0,0.0,0.0,0.0,24098.1
3,648518346341354313,FilteredNeuronSkeleton,50,postsyn,0.0,0.0,0.0,0.0,0.0,95712.3
3,648518346341354496,FilteredNeuronSkeleton,50,postsyn,0.0,0.0,0.0,0.0,0.0,100540.0
3,648518346341355048,FilteredNeuronSkeleton,50,postsyn,0.0,0.0,0.0,0.0,0.0,323800.0
3,648518346341355198,FilteredNeuronSkeleton,50,postsyn,0.0,0.0,0.0,0.0,0.0,29475.2
3,648518346341355736,FilteredNeuronSkeleton,50,postsyn,0.0,0.0,0.0,0.0,0.0,211692.0
3,648518346341356241,FilteredNeuronSkeleton,50,postsyn,0.0,0.0,0.0,0.0,0.0,70524.8
3,648518346341356256,FilteredNeuronSkeleton,50,postsyn,0.0,0.0,0.0,0.0,0.0,1259830.0
3,648518346341357847,FilteredNeuronSkeleton,50,postsyn,0.0,0.0,0.0,184069.0,0.0,0.0
3,648518346341357855,FilteredNeuronSkeleton,50,postsyn,0.0,0.0,0.0,0.0,0.0,238230.0


In [5]:
import time
start_time = time.time()
SkeletonCompartmentLengthWithAxon.populate(reserve_jobs=True)
print(f"Total time = {time.time() - start_time}")

Total time = 177.47842502593994


In [ ]:
pinky.SkeletonCompartmentLength()

In [ ]:
pinky_fc.

In [ ]:
pinky.FullSkeletonContact.heading

In [ ]:
len(pinky.SkeletonContact() & "majority_label_ratio>0.9")

In [ ]:
"""
Level One analysis: Meausre how the neurons proportions match the statistics of the whole volume

Pseudo-code: 
1) Calculate for the volume the total percentage of comparments (but soma)
2) Find the percentage of synapses for the volume
3) For each axon with at least one synapse: 
- Find the compartments they synapsed on
- Weight this vector distribution as a function of length to some power
- Sum all the vectors up and see how close to 0 they can be: 
    a. try different coefficients to see if can make closer to 0 or not
    
"""



In [ ]:
#1) caclulate the total volume's percentages per compartment



In [ ]:
#2) Find axons with only one synapse
